SET VARIABLES

In [1]:
EXPERIMENT_NAME = 'named-entity-recognition_1'
BUCKET = 'ner-project-289117-kubeflowpipelines-default'

IMPORT LIBRARY

In [2]:
import kfp
from kfp import compiler
import kfp.components as comp
import kfp.dsl as dsl
from kfp import gcp

LOAD COMPONENTS

In [17]:
preprocess_operation = comp.load_component_from_url(
    'https://raw.githubusercontent.com/Fitzpatrique/Portfolio/master/examples/named_entity_recognition/components/preprocess/component.yaml')
help(preprocess_operation)

train_operation = kfp.components.load_component_from_url(
    'https://raw.githubusercontent.com/Fitzpatrique/Portfolio/master/examples/named_entity_recognition/components/train/component.yaml')
help(train_operation)

ai_platform_deploy_operation = comp.load_component_from_url(
    'https://raw.githubusercontent.com/Fitzpatrique/Portfolio/master/examples/named_entity_recognition/components/deploy/component.yaml')
help(ai_platform_deploy_operation)

Help on function preprocess:

preprocess(input_1_uri: 'GCSPath', output_x_uri_template: 'GCSPath', output_y_uri_template: 'GCSPath', output_preprocessing_state_uri_template: 'GCSPath')
    preprocess
    Performs the IOB preprocessing.

Help on function train:

train(input_x_uri: 'GCSPath', input_y_uri: 'GCSPath', input_job_dir_uri: 'GCSPath', input_tags: str, input_words: str, input_dropout, output_model_uri_template: 'GCSPath')
    train
    Trains the NER Bi-LSTM.

Help on function deploy:

deploy(model_path: 'GCSPath', model_name: str, model_region: str, model_version: str, model_runtime_version: str, model_prediction_class: str, model_python_version: str, model_package_uris: 'GCSpath')
    deploy
    Deploy the model with custom prediction route



CREATE PIPELINE

In [18]:
@dsl.pipeline(
  name='Named Entity Recognition Pipeline',
  description='Performs preprocessing, training and deployment.'
)
def pipeline():
    
    preprocess_task = preprocess_operation(
        input_1_uri='gs://kubeflow-examples-data/named_entity_recognition_dataset/ner.csv',
        output_y_uri_template="gs://{}/{{workflow.uid}}/preprocess/y/data".format(BUCKET),
        output_x_uri_template="gs://{}/{{workflow.uid}}/preprocess/x/data".format(BUCKET),
        output_preprocessing_state_uri_template="gs://{}/{{workflow.uid}}/model".format(BUCKET)
    ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa')) 
    
    
    train_task = train_operation(
        input_x_uri=preprocess_task.outputs['output_x_uri'],
        input_y_uri=preprocess_task.outputs['output_y_uri'],
        input_job_dir_uri="gs://{}/{{workflow.uid}}/job".format(BUCKET),
        input_tags=preprocess_task.outputs['output_tags'],
        input_words=preprocess_task.outputs['output_words'],
        input_dropout=0.1,
        output_model_uri_template="gs://{}/{{workflow.uid}}/model".format(BUCKET)
    ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa')) 
    
    
    deploy_task = ai_platform_deploy_operation(
        model_path= train_task.output,
        model_name="named_entity_recognition_kubeflow",
        model_region="us-central1",
        model_version="version1",
        model_runtime_version="1.13",
        model_prediction_class="model_prediction.CustomModelPrediction",
        model_python_version="3.5",
        model_package_uris="gs://{}/routine/custom_prediction_routine-0.2.tar.gz".format(BUCKET)
    ).apply(kfp.gcp.use_gcp_secret('user-gcp-sa'))

RUN PIPELINE

In [19]:
pipeline_func = pipeline
#pipeline = kfp.Client().create_run_from_pipeline_func(pipeline, arguments={})

# Run the pipeline on a separate Kubeflow Cluster instead
# (use if your notebook is not running in Kubeflow - e.x. if using AI Platform Notebooks)
pipeline = kfp.Client(host='6e9432b6b27137c1-dot-us-central2.pipelines.googleusercontent.com').create_run_from_pipeline_func(pipeline, arguments={})

/opt/conda/lib/python3.7/site-packages/kfp/components/_data_passing.py:168: UserWarning: Missing type name was inferred as "Float" based on the value "0.1".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))
